In [28]:
import os
import pandas as pd
from glob import glob
import numpy
import fnmatch


In [2]:
raw_beh_path = os.path.abspath(r'C:\Users\USER\Guro_Psy_KJH Dropbox\1.Projects\1_anxiety_VR\3_Data\1_Behavior\0_raw')

In [3]:
# Input: local location for raw behavior data from VRABES
# Output: dictionary, key = sub-xxxx_ses-xx, value = list of path 
# reference: ref1 : 폴더명 생성 규칙 
def gen_dic_paths (raw_beh_path): 
    dic_path = {}  
    subs = os.listdir(raw_beh_path) 
    for sub in subs:
        path2 = os.path.join(raw_beh_path,sub)
        sess = os.listdir(path2) 
        for ses in sess:
            path3 = os.path.join(path2,ses)
            paths = os.listdir(path3) 
            dic_path['_'.join([sub,ses])] = paths           
                
    return dic_path

In [29]:
# Input: return from gen_dic_paths
# output: dictionary, key = sub-xxxx_sex-xx, value = list of filename
# reference: ref2 : Information and location by file

def gen_dic_flist ():
    a = gen_dic_paths(raw_beh_path)
    dic_file = {}    
    for key in a.keys():        
        f_list=[]
        for i in a[key]:
            path_list = [raw_beh_path, key.split('_')[0], key.split('_')[1],i]      
            p = os.path.join(*path_list)
            f_list.extend(glob(p+'/*.csv'))
            f_list.extend(glob(p+'/*.txt'))                      
            
        dic_file[key] = f_list
    return dic_file

In [107]:
def HRV_R_slicer (x):
    eq_ind = x.find('=') # 등호의 index
    name = x[:eq_ind]
    after_eq = x[eq_ind+1:] # 등호 이후의 문자열
    if '(' in after_eq:
        br_ind = after_eq.find('(') # = 이후의 ( 위치 
        val = x[eq_ind+2:eq_ind+br_ind+1] # = 와 ( 사이
    else:
        val = x[eq_ind+2:]
    return name, val

In [99]:
b = gen_dic_flist()
c = b['sub-0001_ses-01'][0]
d = b['sub-0001_ses-01'][1]
e = b['sub-0001_ses-01'][-1]
r_df_c = pd.read_csv(c,header=1)
r_df_d = pd.read_csv(d,header=1)
r_df_e = pd.read_csv(e,header=None)
c_sp = c.split('\\')
col1 = c_sp[-1][:-4]
  

In [110]:
# Input: return from gen_dic_flist
# output: dictionary, key = sub-xxxx_sex-xx, value = dataframe for each experiment

def gen_df ():
    a = gen_dic_flist()
    x = {}
    for key in a.keys():
        print(key)        
        dic_df = {}       
        for i in a[key]:            
            i_sp = i.split('\\')[-1]
            if i_sp[-3:] == 'csv':
                r_df = pd.read_csv(i,header=1)
                col1 = i_sp[:-4]
                if i_sp[-7:-4] =='HRV':
                    col2 = r_df.columns[0]
                    val = r_df.columns[1]
                    dic_df['_'.join([col1,col2])] = val
                else:
                    for d in r_df.columns:
                        val = r_df[d][0]
                        dic_df['_'.join([col1,d])] = val  
            elif i_sp[-3:] == 'txt':
                r_df = pd.read_csv(i,header=None)
                col1 = i_sp[:-4]
                for e in r_df.loc[:,0]:
                    col2, val = HRV_R_slicer(e)
                    dic_df['_'.join([col1,col2])] = val
        x[key] = dic_df
    return x     

In [72]:
ex = gen_df()

sub-0001_ses-01
M04_Base.csv
M04_Base_HRV.csv
M07_S2_P1.csv
M07_S2_P1HRV.csv
M07_S3_P1.csv
M08_S2_P2.csv
M08_S2_P2HRV.csv
M08_S2_P3.csv
M08_S2_P3HRV.csv
M08_S3_P2.csv
M08_S3_P2HRV.csv
M08_S3_P3.csv
M08_S3_P3HRV.csv
M08_S4_P1.csv
M08_S4_P2.csv
M08_S4_P2HRV.csv
M08_S4_P2_1HRV.csv
M08_S4_P3.csv
M08_S4_P3HRV.csv
M08_S4_P3_1HRV.csv
M09_S4_RESP.csv
RawHR.csv
M04_Base_HRV_Result.txt
M07_S2_P1HRV_Result.txt
M08_S2_P2HRV_Result.txt
M08_S2_P3HRV_Result.txt
M08_S3_P2HRV_Result.txt
M08_S3_P3HRV_Result.txt
M08_S4_P2HRV_Result.txt
M08_S4_P2_1HRV_Result.txt
M08_S4_P3HRV_Result.txt
M08_S4_P3_1HRV_Result.txt
sub-0001_ses-02
M04_Base.csv
M04_Base_HRV.csv
M07_S2_P1.csv
M07_S2_P1HRV.csv
M07_S3_P1.csv
M08_S2_P2.csv
M08_S2_P2HRV.csv
M08_S2_P3.csv
M08_S2_P3HRV.csv
M08_S3_P2.csv
M08_S3_P2HRV.csv
M08_S3_P3.csv
M08_S3_P3HRV.csv
M08_S4_P1.csv
M08_S4_P2.csv
M08_S4_P2HRV.csv
M08_S4_P2_1HRV.csv
M08_S4_P3.csv
M08_S4_P3HRV.csv
M08_S4_P3_1HRV.csv
M09_S4_RESP.csv
RawHR.csv
M04_Base_HRV_Result.txt
M07_S2_P1HRV_Result.tx

KeyboardInterrupt: 

In [ ]:
print(b)

실행결과 (Test), a,b의 type에서의 문제는 없음, DictofPath 확인시에는 문제가 없음을 확인
a , b, c = DictofFiles('C:/Users/cabil/Downloads/VRABES_preliminary/1.Behavior/0.raw')

c => 'C:/Users/cabil/Downloads/VRABES_preliminary/1.Behavior/0.raw/sub-0012/ses-01'
실제로 해당 폴더에 behavior 폴더가 2개임
실험 노트 (Note_VRABES_20211020)에 VRABES 정지되어 다시 시작된 기록이 있음.
-> 이어서 수행된 파일들이 생성되었음
-> 기록에 남기고 한 폴더로 정리할 예정 (기록 양식 만들 예정)


#### 1.2 Func 2 Behavior 데이터가 제대로 생성되지 않은 실험 확인 

In [3]:
# Input: DictofFiles 함수의 DictofFiles 값과, reference로 삼을 실험
# Output: Dictionary, keys: sub_ses, values: reference에 있으나 해당 실험에 없는 파일의 리스트

def NdstoCh (path, r_sub_ses): # Needs to check, 확인이 필요한 sub_ses 찾기위한 함수
    D_File, b, c = DictofFiles(path)    
    Ref = D_File[r_sub_ses] 
    NdstoCh = {}
    for k in D_File.keys():
        NdstoCh[k] = [x for x in Ref if x not in D_File[k]] # reference에 있고, 해당에는 없는 것을 찾는 방식
    return NdstoCh 

a = NdstoCh('C:/Users/cabil/Downloads/VRABES_preliminary/1.Behavior/0.raw','sub-0001_ses-01')
실제로 sub-0001_ses-01 이 한개를 제외한 모든 파일이 생성됨

실행결과 a 결과에서 이상이 있는 실험 확인함 <br>
-> 해당 실험의 노트내용 확인함<br>
sub-0007_ses-03 -> 불안군 / ses-04까지 마무리함 <br>
sub-0008_ses-01 -> 대조군 / ses-02까지 마무리함, ses-02데이터 활용 가능 <br>
sub-0009_ses-03 -> 불안군 / ses-04까지 마무리함<br>
sub-0010_ses-01 -> 불안군, 중도탈락 환자 / ses-03 까지 진행<br>
sub-0011_ses-01 -> 대조군 / ses-02까지 마무리함, ses-02 데이터 활용 가능 <br>
sub-0013_ses-01 -> 대조군 / ses-02까지 마무리함, ses-02 데이터 활용 가능 <br>
sub-0019_ses-01 -> 불안군, 중도탈락 환자 / ses-04 까지 진행 <br>
sub-0037_ses-01 -> 불안군, <br>
sub-0042_ses-01 -> 불안군, <br>

Baseline 분석에서 문제가 되는 데이터는 (대조군은 ses-02 데이터 활용) 아래 4명임. <br>
sub-0010_ses-01 -> 실험노트 내용 참조하여 포함할지 논의가 필요<br>
sub-0019_ses-01 -> HRV 데이터만 누락 -> time stamp정보가 없음,포함할지 논의가 필요 <br>
sub-0037_ses-01 -> 불안군, => ses01만 진행된 환자들로 다시 시행하는 것을 고려 중<br>
sub-0042_ses-01 -> 불안군, => // 우선 이번 분석에서는 제외<br>



================================================================================================================
파일리스트와 해당 실험의 실험노트 내용<br>

{ 'sub-0007_ses-03': ['M04_Base_HRV.csv',  'M04_Base_HRV_Result.txt',  'M07_S2_P1HRV.csv',
  'M07_S2_P1HRV_Result.txt',  'M08_S2_P2HRV.csv',  'M08_S2_P2HRV_Result.txt',  'M08_S2_P3HRV.csv',
  'M08_S2_P3HRV_Result.txt',  'M08_S3_P2HRV.csv',  'M08_S3_P2HRV_Result.txt',  'M08_S3_P3HRV.csv',
  'M08_S3_P3HRV_Result.txt',  'M08_S4_P2HRV.csv',  'M08_S4_P2HRV_Result.txt',  'M08_S4_P2_1HRV.csv',
  'M08_S4_P2_1HRV_Result.txt',  'M08_S4_P3HRV.csv',  'M08_S4_P3HRV_Result.txt',  'M08_S4_P3_1HRV.csv',
  'M08_S4_P3_1HRV_Result.txt',  'RawHR.csv'],
  
  특이 사항 없음으로 기록됨 <br>
  
 'sub-0008_ses-01': ['M04_Base.csv',  'M04_Base_HRV.csv',  'M04_Base_HRV_Result.txt',  'M07_S2_P1.csv',
  'M07_S3_P1.csv',  'M08_S2_P2.csv',  'M08_S2_P2HRV_Result.txt',  'M08_S2_P3.csv',  'M08_S2_P3HRV.csv',
  'M08_S2_P3HRV_Result.txt',  'M08_S3_P2.csv',  'M08_S3_P2HRV.csv',  'M08_S3_P2HRV_Result.txt',
  'M08_S3_P3.csv',  'M08_S3_P3HRV.csv',  'M08_S3_P3HRV_Result.txt',  'M08_S4_P1.csv',  'M08_S4_P2.csv',
  'M08_S4_P2HRV.csv',  'M08_S4_P2HRV_Result.txt',  'M08_S4_P2_1HRV.csv',  'M08_S4_P2_1HRV_Result.txt',
  'M08_S4_P3.csv',  'M08_S4_P3HRV.csv',  'M08_S4_P3HRV_Result.txt',  'M08_S4_P3_1HRV.csv',
  'M08_S4_P3_1HRV_Result.txt',  'M09_S4_RESP.csv',  'RawHR.csv'],
  
  특이 사항 없음으로 기록됨 <br>
  
 'sub-0009_ses-03': ['M08_S4_P3.csv',  'M08_S4_P3HRV_Result.txt',  'M08_S4_P3_1HRV.csv',
  'M08_S4_P3_1HRV_Result.txt',  'M09_S4_RESP.csv'],
  
  • 자극감응훈련에서 머리들어올리기를 시작해서 5~6초후 오큐러스가 꺼져버려서 VRABES를 종료함 -> missing data로의 고려가 필요함
  • 투약중단상태<br>
  
 'sub-0010_ses-01': ['M08_S4_P2.csv',  'M08_S4_P3.csv',  'M08_S4_P3HRV.csv',  'M08_S4_P3HRV_Result.txt',
  'M08_S4_P3_1HRV.csv',  'M08_S4_P3_1HRV_Result.txt',  'M09_S4_RESP.csv'],
  
  ->  프로그램 시작전부터 불편한 듯 가슴에 손을 대는 행동을 여러 번 보임.
모듈3 08-S3-a : '머리아프다'호소 // 모듈3 08-S3-b-E '머리아프다'호소 // 모듈4 06-S4 '어지럽다'면서 고개를 숙이고 
모듈4 08-S4-a   힘들다고 고개를 숙이고 못하겠다 머리아프다며 그만하자고함. -> 조기 중단한 환자 <br>

  
  
 'sub-0011_ses-01': ['M04_Base.csv',  'M04_Base_HRV.csv',  'M04_Base_HRV_Result.txt',  'M07_S2_P1.csv',
  'M07_S3_P1.csv',  'M08_S2_P2.csv',  'M08_S2_P2HRV_Result.txt',  'M08_S2_P3.csv',  'M08_S2_P3HRV.csv',
  'M08_S2_P3HRV_Result.txt',  'M08_S3_P2.csv',  'M08_S3_P2HRV.csv',  'M08_S3_P2HRV_Result.txt',
  'M08_S3_P3.csv',  'M08_S3_P3HRV.csv',  'M08_S3_P3HRV_Result.txt',  'M08_S4_P1.csv',  'M08_S4_P2.csv',
  'M08_S4_P2HRV.csv',  'M08_S4_P2HRV_Result.txt',  'M08_S4_P2_1HRV.csv',  'M08_S4_P2_1HRV_Result.txt',
  'M08_S4_P3.csv',  'M08_S4_P3HRV.csv',  'M08_S4_P3HRV_Result.txt',  'M08_S4_P3_1HRV.csv',  'M08_S4_P3_1HRV_Result.txt',  'M09_S4_RESP.csv',  'RawHR.csv'],
  
   특이 사항 없음으로 기록됨 <br>
  
  'sub-0013_ses-01': ['M04_Base_HRV.csv',  'M04_Base_HRV_Result.txt',  'M07_S2_P1HRV.csv',
  'M07_S2_P1HRV_Result.txt',  'M08_S2_P2HRV.csv',  'M08_S2_P2HRV_Result.txt',  'M08_S2_P3HRV.csv',
  'M08_S2_P3HRV_Result.txt',  'M08_S3_P2HRV.csv',  'M08_S3_P2HRV_Result.txt',  'M08_S3_P3HRV.csv',
  'M08_S3_P3HRV_Result.txt',  'M08_S4_P2HRV.csv',  'M08_S4_P2HRV_Result.txt',  'M08_S4_P2_1HRV.csv',
  'M08_S4_P2_1HRV_Result.txt',  'M08_S4_P3HRV.csv',  'M08_S4_P3HRV_Result.txt',  'M08_S4_P3_1HRV.csv',
  'M08_S4_P3_1HRV_Result.txt',  'RawHR.csv'],
  
   특이 사항 없음으로 기록됨<br>
  
 'sub-0019_ses-01': ['M08_S4_P3HRV_Result.txt'],
 
   특이 사항 없음으로 기록됨 <br>
 
 'sub-0037_ses-01': ['desktop.ini',  'M04_Base_HRV.csv',  'M04_Base_HRV_Result.txt',  'M07_S2_P1HRV.csv',
  'M07_S2_P1HRV_Result.txt',  'M08_S2_P2HRV.csv',  'M08_S2_P2HRV_Result.txt',  'M08_S2_P3HRV.csv',
  'M08_S2_P3HRV_Result.txt',  'M08_S3_P2HRV.csv',  'M08_S3_P2HRV_Result.txt',  'M08_S3_P3HRV.csv',
  'M08_S3_P3HRV_Result.txt',  'M08_S4_P2HRV.csv',  'M08_S4_P2HRV_Result.txt',  'M08_S4_P2_1HRV.csv',
  'M08_S4_P2_1HRV_Result.txt',  'M08_S4_P3HRV.csv',  'M08_S4_P3HRV_Result.txt',  'M08_S4_P3_1HRV.csv',
  'M08_S4_P3_1HRV_Result.txt',  'RawHR.csv'],
  
   특이 사항 없음으로 기록됨 <br>
  
 'sub-0042_ses-01': ['desktop.ini',  'M04_Base_HRV.csv',  'M04_Base_HRV_Result.txt',  'M07_S2_P1HRV.csv',
  'M07_S2_P1HRV_Result.txt',  'M08_S2_P2HRV.csv',  'M08_S2_P2HRV_Result.txt',  'M08_S2_P3HRV.csv',
  'M08_S2_P3HRV_Result.txt',  'M08_S3_P2HRV.csv',  'M08_S3_P2HRV_Result.txt',  'M08_S3_P3HRV.csv',
  'M08_S3_P3HRV_Result.txt',  'M08_S4_P2HRV.csv',  'M08_S4_P2HRV_Result.txt',  'M08_S4_P2_1HRV.csv',
  'M08_S4_P2_1HRV_Result.txt',  'M08_S4_P3HRV.csv',  'M08_S4_P3HRV_Result.txt',  'M08_S4_P3_1HRV.csv',
  'M08_S4_P3_1HRV_Result.txt',  'RawHR.csv'],
  
  VRABES 프로그램을 실행하여 대상자가 마지막 프로그램 종료를 눌렀으나 VRABES가 종료되지 않았고 
프로그램 종료라는 마지막 화면이 계속 로딩중이었음 -> 프로그램 문제로 missing으로 보임. 

#### 1.3 Func 3 Behavior data (VR 관련 행동 데이터) dictionary 생성
##### 한개의 실험 (sub_ses)에 대해서 dictionary를 생성
##### Behavior data가 있는 파일 list
files = ['M04_Base.csv','M07_S2_P1.csv','M08_S2_P2.csv','M08_S2_P3.csv','M07_S3_P1.csv','M08_S3_P2.csv',\
         'M08_S3_P3.csv','M08_S4_P1.csv','M08_S4_P2.csv','M08_S4_P3.csv','M09_S4_RESP.csv']




###### columns들은 규칙 참조 
cols = ['id','AS_b','AS_PrEx','Prg_2','AS_Ex2','Prg_6','AS_PoEx','AS_PrRlx','AS_PoRR','AS_PoMR',\
           'AS_PrInt','Dsc_HV','AS_PoHV','Dsc_HD','AS_PoHD','S_Int']
###### 참조해야하는 파일명들
Behav_F_list = ['M04_Base.csv','M07_S2_P1.csv','M08_S2_P2.csv','M08_S2_P3.csv','M07_S3_P1.csv','M08_S3_P2.csv',\
         'M08_S3_P3.csv','M08_S4_P1.csv','M08_S4_P2.csv','M08_S4_P3.csv','M09_S4_RESP.csv']

In [4]:
# Input: path, 해당 실험 (ex), 
# Output:  dictionary, key는 cols, id는 해당 실험, values는 해당 파일별 규칙, 없으면 np.nan
# cols / files의 내용을 함수내에서 그냥 활용하는 형태로 작성함 -> 개선?

def G_behav_df (path, ex):
    files, paths, c = DictofFiles(path)
    T_F_list = files[ex] # 확인하고자 하는 실험의 filelist
    T_path = paths[ex] # 확인하고자 하는 실험의 path
    behav_dic = {} # dafaframe 생성용 dictionary
    behav_dic['id'] = ex
    
    if 'M04_Base.csv' in T_F_list:            
        df = pd.read_csv(T_path+'/'+'M04_Base.csv', header=1) 
        AS_b = df.iloc[0,0]
        behav_dic['AS_b'] = AS_b  
    else: behav_dic['AS_b'] = np.nan

    if 'M07_S2_P1.csv' in T_F_list:
        df = pd.read_csv(T_path+'/'+'M07_S2_P1.csv', header=1)  
        AS_PrEx = df.iloc[0,0]
        Prg_2 = df.iloc[0,3]
        behav_dic['AS_PrEx'] = AS_PrEx
        behav_dic['Prg_2'] = Prg_2    
    else: 
        behav_dic['AS_PrEx'] = np.nan
        behav_dic['Prg_2'] = np.nan 
        
    if 'M08_S2_P2.csv' in T_F_list:
        df = pd.read_csv(T_path+'/'+'M08_S2_P2.csv', header=1)           
        AS_Ex2 = df.iloc[0,0]
        Prg_6 = df.iloc[0,3]
        behav_dic['AS_Ex2'] = AS_Ex2
        behav_dic['Prg_6'] = Prg_6
    else:
        behav_dic['AS_Ex2'] = np.nan
        behav_dic['Prg_6'] = np.nan
        
    if 'M08_S2_P2.csv' in T_F_list:
        df = pd.read_csv(T_path+'/'+'M08_S2_P2.csv', header=1)           
        AS_Ex2 = df.iloc[0,0]
        Prg_6 = df.iloc[0,3]
        behav_dic['AS_Ex2'] = AS_Ex2
        behav_dic['Prg_6'] = Prg_6  
    else:
        behav_dic['AS_Ex2'] = np.nan
        behav_dic['Prg_6'] = np.nan
    
    if 'M08_S2_P3.csv' in T_F_list:
        df = pd.read_csv(T_path+'/'+'M08_S2_P3.csv', header=1)           
        AS_PoEx = df.iloc[0,0]
        behav_dic['AS_PoEx'] = AS_PoEx
    else:behav_dic['AS_PoEx'] = np.nan
    
    if 'M07_S3_P1.csv' in T_F_list:
        df = pd.read_csv(T_path+'/'+'M07_S3_P1.csv', header=1)            
        AS_PrRlx = df.iloc[0,0]
        behav_dic['AS_PrRlx'] = AS_PrRlx
    else: behav_dic['AS_PrRlx'] = np.nan
    
    if 'M08_S3_P2.csv' in T_F_list:
        df = pd.read_csv(T_path+'/'+'M08_S3_P2.csv', header=1)            
        AS_PoRR = df.iloc[0,0]
        behav_dic['AS_PoRR'] = AS_PoRR
    else:behav_dic['AS_PoRR'] = np.nan
        
    if 'M08_S3_P3.csv' in T_F_list:
        df = pd.read_csv(T_path+'/'+'M08_S3_P3.csv', header=1)            
        AS_PoMR = df.iloc[0,0]
        behav_dic['AS_PoMR'] = AS_PoMR
    else: behav_dic['AS_PoMR'] = np.nan
    
    if 'M08_S4_P1.csv' in T_F_list:
        df = pd.read_csv(T_path+'/'+'M08_S4_P1.csv', header=1)            
        AS_PrInt = df.iloc[0,0]
        behav_dic['AS_PrInt'] = AS_PrInt
    else: behav_dic['AS_PrInt'] = np.nan
    
    if 'M08_S4_P2.csv' in T_F_list:
        df = pd.read_csv(T_path+'/'+'M08_S4_P2.csv', header=1)            
        Dsc_HV = df.iloc[0,0]
        AS_PoHV = df.iloc[0,3]
        behav_dic['Dsc_HV'] = Dsc_HV
        behav_dic['AS_PoHV'] = AS_PoHV
    else: 
        behav_dic['Dsc_HV'] = np.nan
        behav_dic['AS_PoHV'] = np.nan
        
    if 'M08_S4_P3.csv' in T_F_list:
        df = pd.read_csv(T_path+'/'+'M08_S4_P3.csv', header=1)           
        Dsc_HD = df.iloc[0,0]
        AS_PoHD = df.iloc[0,3]
        behav_dic['Dsc_HD'] = Dsc_HD
        behav_dic['AS_PoHD'] = AS_PoHD
    else: 
        behav_dic['Dsc_HD'] = np.nan
        behav_dic['AS_PoHD'] = np.nan
        
    if 'M09_S4_RESP.csv' in T_F_list:
        df = pd.read_csv(T_path+'/'+'M09_S4_RESP.csv', header=1)            
        S_Int = df.iloc[0,0]
        behav_dic['S_Int'] = S_Int
    else:behav_dic['S_Int'] = np.nan
        
    return behav_dic



실행결과
path = 'C:/Users/cabil/Downloads/VRABES_preliminary/1.Behavior/0.raw'
a = G_behav_df(path,'sub-0001_ses-01' )
{'id': 'sub-0001_ses-01', 'AS_b': 80, 'AS_PrEx': 10, 'Prg_2': True, 'AS_Ex2': 10, 'Prg_6': True, 'AS_PoEx': 0,
 'AS_PrRlx': 10, 'AS_PoRR': 0, 'AS_PoMR': 0, 'AS_PrInt': 20, 'Dsc_HV': False, 'AS_PoHV': 20, 'Dsc_HD': False,
 'AS_PoHD': 10, 'S_Int': '가슴두근거림'}


#### 1.4 Func 4 HRV data (UBpulse 생성 데이터) dictionary 생성
##### 한개의 실험 (sub_ses)에 대해서 dictionary를 생성
##### HRV data는 두 종료의 파일, HRV.csv 와 HRV_Result.csv 두 종류 // Time 관련 정보는 HRV.csv 에서 결과들은 HRV_Result.csv 에서  
##### 두 함수를 나누어서 만드는 것으로 


#### 1.4.1 Func 4.1 HRV_Time dictionary 생성

1. HRV Time data
  * M04_Base_HRV.csv > 시작시점, 가상환경 전 <br>
  Start Time = ST_base<br>
  
  * M07_S2_P1HRV.csv > 엘레베이터 노출 전 <br>
  Start Time = ST_PrEx<br>
  
  * M08_S2_P2HRV.csv > 엘레베이터 2명 노출 <br>
  Start Time = ST_Ex2  <br>
  
  * M08_S2_P3HRV.csv > 엘레베이터 6명 노출  <br>
  Start Time = ST_Ex6  <br>
  
  * M08_S3_P2HRV.csv > 호흡이완 <br>  
  Start Time = ST_RR  <br>
  
  * M08_S3_P3HRV.csv > 근육이완 <br>
  Start Time = ST_MR  <br>
  
  * M08_S4_P2HRV.csv > 과호흡 종료 후 <br>
  Start Time = ST_HV <br>
  
  * M08_S4_P3HRV.csv > 머리숙이기 종료 후 <br>
  Start Time = ST_HD <br>

columns들은 규칙 참조
cols = ['id','ST_base','ST_PrEx', 'ST_Ex2', 'ST_Ex6', 'ST_RR', 'ST_MR', 'ST_HV', 'ST_HD']

참조해야하는 파일명들
Behav_F_list = ['M04_Base_HRV.csv','M07_S2_P1HRV.csv','M08_S2_P2HRV.csv','M08_S2_P3HRV.csv','M08_S3_P2HRV.csv',
'M08_S3_P3HRV.csv','M08_S4_P2HRV.csv','M08_S4_P3HRV.csv']

In [5]:
# Input: path, ex
# Output: dictionary, keys: 변수명, values는 id는 해당 실험 및 해당 파일별 규칙, 없으면 np.nan

def G_HRV_Time_dic (path, ex):
    files, paths, c = DictofFiles(path)
    T_F_list = files[ex] # 확인하고자 하는 실험의 filelist
    T_path = paths[ex] # 확인하고자 하는 실험의 path
    HRV_Time_dic = {}
    HRV_Time_dic['id'] = ex
    
    if 'M04_Base_HRV.csv' in T_F_list:            
        df = pd.read_csv(T_path+'/'+'M04_Base_HRV.csv')          
        ST_base = df.iloc[0,0]        
        HRV_Time_dic['ST_base'] = ST_base           
    else: HRV_Time_dic['ST_base'] = np.nan
    
    if 'M07_S2_P1HRV.csv' in T_F_list:
        df = pd.read_csv(T_path+'/'+'M07_S2_P1HRV.csv')
        ST_PrEx = df.iloc[0,0]
        HRV_Time_dic['ST_PrEx'] = ST_PrEx
    else: HRV_Time_dic['ST_PrEx'] = np.nan
    
    if 'M08_S2_P2HRV.csv' in T_F_list:
        df = pd.read_csv(T_path+'/'+'M08_S2_P2HRV.csv')
        ST_Ex2 = df.iloc[0,0]
        HRV_Time_dic['ST_Ex2'] = ST_Ex2
    else: HRV_Time_dic['ST_Ex2'] = np.nan
                          
    if 'M08_S2_P3HRV.csv' in T_F_list:
        df = pd.read_csv(T_path+'/'+'M08_S2_P3HRV.csv')
        ST_Ex6 = df.iloc[0,0]
        HRV_Time_dic['ST_Ex6'] = ST_Ex6
    else: HRV_Time_dic['ST_Ex6'] = np.nan
                                                   
    if 'M08_S3_P2HRV.csv' in T_F_list:
        df = pd.read_csv(T_path+'/'+'M08_S3_P2HRV.csv')
        ST_RR = df.iloc[0,0]
        HRV_Time_dic['ST_RR'] = ST_RR
    else: HRV_Time_dic['ST_RR'] = np.nan
        
    if 'M08_S3_P3HRV.csv' in T_F_list:
        df = pd.read_csv(T_path+'/'+'M08_S3_P3HRV.csv')
        ST_MR = df.iloc[0,0]
        HRV_Time_dic['ST_MR'] = ST_MR
    else: HRV_Time_dic['ST_MR'] = np.nan
        
    if 'M08_S4_P2HRV.csv' in T_F_list:
        df = pd.read_csv(T_path+'/'+'M08_S4_P2HRV.csv')
        ST_HV = df.iloc[0,0]
        HRV_Time_dic['ST_HV'] = ST_HV
    else: HRV_Time_dic['ST_HV'] = np.nan        
    
    if 'M08_S4_P3HRV.csv' in T_F_list:
        df = pd.read_csv(T_path+'/'+'M08_S4_P3HRV.csv')
        ST_HD = df.iloc[0,0]
        HRV_Time_dic['ST_HD'] = ST_HD
    else: HRV_Time_dic['ST_HD'] = np.nan  

    return HRV_Time_dic

path = 'C:/Users/cabil/Downloads/VRABES_preliminary/1.Behavior/0.raw' <br>
a = G_HRV_Time_dic(path,'sub-0001_ses-01' )<br>
실행결과<br>
{'id': 'sub-0001_ses-01',
 'ST_base': '11:40:41.678',
 'ST_PrEx': '11:44:03.142',
 'ST_Ex2': '11:46:47.579',
 'ST_Ex6': '11:49:28.663',
 'ST_RR': '11:52:49.847',
 'ST_MR': '11:55:26.736',
 'ST_HV': '11:59:31.197',
 'ST_HD': '12:02:55.703'}

#### 1.4.2 Func 4.2 HRV_Result dictionary 생성
##### 현재 HRV_Result는 txt 파일로 txt를 가져와야 함
##### return 값 구조는 dictionary 안에 결과의 list 로 생성 -> Results 파일안의 순서는 동일하므로
 line마다의 형식은 Ts = 117.25 또는 deltaTime = 0.2500 (s) 의 형태 숫자만 가져오는 것으로 <br>
 '(' 이 이 '=' 오른쪽에 있으면 -> '='과 '(' 사이를 slice <br>
 없는 경우 '=' 부터 끝까지 slice 하는 방식으로  <br>
 우선은 string으로 가져오기로, HR Max-Min = 79.893-62.048 (bpm)에서 문제가 있음  <br>

#### 규칙을 정리
<h3> 규칙 </h3>
1. HRV data
  * M04_Base_HRV_Result.txt >시작시점, 가상환경 전 <br>
  HRV_R_base   <br>
  * M07_S2_P1HRV_Result.txt > 엘레베이터 노출 전 <br>
  HRV_R_PrEx   <br>
  * M08_S2_P2HRV_Result.txt > 엘레베이터 2명 노출  <br>
  HRV_R_Ex2   <br>
  * M08_S2_P3HRV_Result.txt > 엘레베이터 6명 노출  <br>
  HRV_R__Ex6  <br>
  * M08_S3_P2HRV_Result.txt<br>
  HRV_R_RR  <br>
  * M08_S3_P3HRV_Result.txt<br>
  HRV_R_MR  <br>
  * M08_S4_P2HRV_Result.txt<br>
  HRV_R_HV  <br>
  * M08_S4_P3HRV_Result.txt<br>
  HRV_R_HD<br>


columns들은 규칙 참조<br>
cols = ['id','HRV_R_base','HRV_R_PrEx', 'HRV_R_Ex2', 'HRV_R_Ex6', 'HRV_R_RR', 'HRV_R_MR', 'HRV_R_HV', 'HRV_R_HD']<br>

참조해야하는 파일명들<br>
Behav_F_list = ['M04_Base_HRV_Result.txt','M07_S2_P1HRV_Result.txt','M08_S2_P2HRV_Result.txt',
'M08_S2_P3HRV_Result.txt','M08_S3_P2HRV_Result.txt','M08_S3_P3HRV_Result.txt',
'M08_S4_P2HRV_Result.txt','M08_S4_P3HRV_Result.txt']

In [9]:
path = 'C:/Users/cabil/Downloads/VRABES_preliminary/1.Behavior/0.raw'
ex = 'sub-0001_ses-01'
files, paths, c = DictofFiles(path)
T_F_list = files[ex] # 확인하고자 하는 실험의 filelist
T_path = paths[ex] # 확인하고자 하는 실험의 path
df = pd.read_csv(T_path+'/'+'M04_Base_HRV_Result.txt',header=None)      
df

,0
0,T = 117.344
1,Ts = 117.25
2,Nt = 470
3,Nf = 512
4,Fs = 4 (Hz)
5,deltaTime = 0.2500 (s)
6,deltaFreq = 0.000977 (Hz)
7,Mean RR = 856.833 (ms)
8,SDNN = 42.505 (ms)
9,SDSD = 20.891 (ms)


line마다의 형식은 Ts = 117.25 또는 deltaTime = 0.2500 (s) 의 형태 숫자만 가져오는 것으로 <br>
'(' 이 이 '=' 오른쪽에 있으면 -> '='과 '(' 사이를 slice<br>
없는 경우 '=' 부터 끝까지 slice 하는 방식으로<br>
우선은 string으로 가져오기로, HR Max-Min = 79.893-62.048 (bpm)에서 문제가 있음<br>
=> 이 부분에 대한 함수를 만들어서
=> df에 apply 하는 형태로 작성

In [25]:
def HRV_R_slicer (x):
    eq_ind = x.find('=') # 등호의 index
    after_eq = x[eq_ind+1:] # 등호 이후의 문자열
    if '(' in after_eq:
        br_ind = after_eq.find('(') # = 이후의 ( 위치 
        val = x[eq_ind+2:eq_ind+br_ind+1] # = 와 ( 사이
    else:
        val = x[eq_ind+2:]
    return val

In [30]:
df[1]=df[0].apply(HRV_R_slicer)
df # 제대로 되는 것을 확임함

,0,1,2
0,T = 117.344,117.344,37
1,Ts = 117.25,117.25,37
2,Nt = 470,470,37
3,Nf = 512,512,37
4,Fs = 4 (Hz),4,37
5,deltaTime = 0.2500 (s),0.2500,37
6,deltaFreq = 0.000977 (Hz),0.000977,37
7,Mean RR = 856.833 (ms),856.833,37
8,SDNN = 42.505 (ms),42.505,37
9,SDSD = 20.891 (ms),20.891,37


In [31]:
# Input: path, ex
# Output: dictionary, keys: cols 안에, values는 id는 해당 실험 및 해당파일 내 value들의 series, 파일이 없으면 np.nan

def G_HRV_R_dic (path, ex):
    files, paths, c = DictofFiles(path)
    T_F_list = files[ex] # 확인하고자 하는 실험의 filelist
    T_path = paths[ex] # 확인하고자 하는 실험의 path
    HRV_R_dic = {}
    HRV_R_dic['id'] = ex
    # index 미리 지정해서 추가해두기 
    
    if 'M04_Base_HRV_Result.txt' in T_F_list:            
        df = pd.read_csv(T_path+'/'+'M04_Base_HRV_Result.txt',header=None)          
        df['val'] = df[0].apply(HRV_R_slicer)                
        HRV_R_base = df['val']        
        HRV_R_dic['HRV_R_base'] = HRV_R_base           
    else: HRV_R_dic['HRV_R_base'] = np.nan
    
    if 'M07_S2_P1HRV_Result.txt' in T_F_list:            
        df = pd.read_csv(T_path+'/'+'M07_S2_P1HRV_Result.txt',header=None)          
        df['val'] = df[0].apply(HRV_R_slicer)                
        HRV_R_PrEx = df['val']        
        HRV_R_dic['HRV_R_PrEx'] = HRV_R_PrEx           
    else: HRV_R_dic['HRV_R_PrEx'] = np.nan
        
    if 'M08_S2_P2HRV_Result.txt' in T_F_list:            
        df = pd.read_csv(T_path+'/'+'M08_S2_P2HRV_Result.txt',header=None)          
        df['val'] = df[0].apply(HRV_R_slicer)                
        HRV_R_Ex2 = df['val']        
        HRV_R_dic['HRV_R_Ex2'] = HRV_R_Ex2           
    else: HRV_R_dic['HRV_R_Ex2'] = np.nan
    
    if 'M08_S2_P3HRV_Result.txt' in T_F_list:            
        df = pd.read_csv(T_path+'/'+'M08_S2_P3HRV_Result.txt',header=None)          
        df['val'] = df[0].apply(HRV_R_slicer)                
        HRV_R_Ex6 = df['val']        
        HRV_R_dic['HRV_R_Ex6'] = HRV_R_Ex6           
    else: HRV_R_dic['HRV_R_Ex6'] = np.nan
    
    if 'M08_S3_P2HRV_Result.txt' in T_F_list:            
        df = pd.read_csv(T_path+'/'+'M08_S3_P2HRV_Result.txt',header=None)          
        df['val'] = df[0].apply(HRV_R_slicer)                
        HRV_R_RR = df['val']        
        HRV_R_dic['HRV_R_RR'] = HRV_R_RR           
    else: HRV_R_dic['HRV_R_RR'] = np.nan
        
    if 'M08_S4_P2HRV_Result.txt' in T_F_list:            
        df = pd.read_csv(T_path+'/'+'M08_S4_P2HRV_Result.txt',header=None)          
        df['val'] = df[0].apply(HRV_R_slicer)                
        HRV_R_HV = df['val']        
        HRV_R_dic['HRV_R_HV'] = HRV_R_HV           
    else: HRV_R_dic['HRV_R_HV'] = np.nan
    
    if 'M08_S4_P3HRV_Result.txt' in T_F_list:            
        df = pd.read_csv(T_path+'/'+'M08_S4_P3HRV_Result.txt',header=None)          
        df['val'] = df[0].apply(HRV_R_slicer)                
        HRV_R_HD = df['val']        
        HRV_R_dic['HRV_R_HD'] = HRV_R_HD           
    else: HRV_R_dic['HRV_R_HD'] = np.nan
            
    return HRV_R_dic

In [32]:
path = 'C:/Users/cabil/Downloads/VRABES_preliminary/1.Behavior/0.raw'
a = G_HRV_R_dic(path,'sub-0001_ses-01' )
a


{'id': 'sub-0001_ses-01',
 'HRV_R_base': 0            117.344
 1             117.25
 2                470
 3                512
 4                 4 
 5            0.2500 
 6          0.000977 
 7           856.833 
 8            42.505 
 9            20.891 
 10           70.198 
 11            3.506 
 12    79.893-62.048 
 13          41.0419 
 14               100
 15          72.9927 
 16                35
 17          25.5475 
 18           11.5000
 19         179.6875 
 20            10.268
 21         1968.150 
 22            0.000 
 23          262.327 
 24         1222.531 
 25          483.292 
 26            0.024 
 27            0.063 
 28            0.389 
 29           2.5296 
 30             7.109
 31             6.181
 32               7.4
 33             71.7 
 34             28.3 
 35             53.5 
 36             46.5 
 Name: val, dtype: object,
 'HRV_R_PrEx': 0             120.77
 1             120.75
 2                484
 3                512
 4               

### 2. 데이터 생성

#### 2.1 baseline 대상자 선정
##### DictofFiles 첫번째 resturn 값에서 keys를 활용해서 이후 함수의 ex 를 고르는 방식으로 
##### 대상이 되는 ex의 list를 생성해야함, 아래의 규칙에 따라서 

###### Baseline 분석에서 문제가 되는 데이터는 (대조군은 ses-02 데이터 활용) 아래 4명임. <br>
sub-0010_ses-01 -> 실험노트 내용 참조하여 포함할지 논의가 필요-> 제외 <br> 
sub-0019_ses-01 -> HRV 데이터만 누락 -> time stamp정보가 없음,포함할지 논의가 필요 -> 제외 <br>
sub-0037_ses-01 -> 불안군, => ses01만 진행된 환자들로 다시 시행하는 것을 고려 중-> 제외<br> 
sub-0042_ses-01 -> 불안군, => // 우선 이번 분석에서는 제외-> 제외<br> 
sub-0008_ses-01 -> 대조군 / ses-02까지 마무리함, ses-02 데이터 활용 가능 <br>
sub-0011_ses-01 -> 대조군 / ses-02까지 마무리함, ses-02 데이터 활용 가능 <br>
sub-0013_ses-01 -> 대조군 / ses-02까지 마무리함, ses-02 데이터 활용 가능 <br>

###### ses-01 고르고 -> sub-0010, 0019, 0037, 0042 제외하고 -> ses-02 고르고, sub-0008, 0011, 0013 만 추가 <br>

In [40]:
T_list = ['sub-0001_ses-01', 'sub-0002_ses-01','sub-0003_ses-01','sub-0004_ses-01', 'sub-0005_ses-01',
          'sub-0006_ses-01', 'sub-0007_ses-01','sub-0008_ses-02','sub-0009_ses-01', 'sub-0011_ses-02',
          'sub-0013_ses-02', 'sub-0014_ses-01','sub-0015_ses-01','sub-0016_ses-01', 'sub-0017_ses-01',
          'sub-0018_ses-01', 'sub-0020_ses-01','sub-0021_ses-01','sub-0022_ses-01', 'sub-0023_ses-01',
          'sub-0024_ses-01', 'sub-0025_ses-01','sub-0026_ses-01','sub-0027_ses-01', 'sub-0028_ses-01',
          'sub-0029_ses-01', 'sub-0030_ses-01','sub-0031_ses-01','sub-0032_ses-01', 'sub-0033_ses-01',
          'sub-0034_ses-01', 'sub-0035_ses-01','sub-0036_ses-01','sub-0038_ses-01', 'sub-0039_ses-01',
          'sub-0040_ses-01', 'sub-0041_ses-01','sub-0043_ses-01','sub-0044_ses-01']

In [41]:
path = 'C:/Users/cabil/Downloads/VRABES_preliminary/1.Behavior/0.raw'

Behav_df = pd.DataFrame()
for i in T_list:
    a = G_behav_df (path, i)
    Behav_df = Behav_df.append(a,ignore_index=True)
    Behav_df

In [42]:
Behav_df

,id,AS_b,AS_PrEx,Prg_2,AS_Ex2,Prg_6,AS_PoEx,AS_PrRlx,AS_PoRR,AS_PoMR,AS_PrInt,Dsc_HV,AS_PoHV,Dsc_HD,AS_PoHD,S_Int
0,sub-0001_ses-01,80.0,10.0,1.0,10.0,1.0,0.0,10.0,0.0,0.0,20.0,0.0,20.0,0.0,10.0,가슴두근거림
1,sub-0002_ses-01,0.0,0.0,True,0.0,True,0.0,0.0,0.0,0.0,0.0,False,0.0,False,0.0,증상없음
2,sub-0003_ses-01,0.0,0.0,True,0.0,True,0.0,0.0,0.0,0.0,0.0,False,0.0,False,0.0,증상없음
3,sub-0004_ses-01,0.0,0.0,True,0.0,True,0.0,0.0,0.0,0.0,0.0,False,0.0,False,0.0,증상없음
4,sub-0005_ses-01,0.0,0.0,True,0.0,True,0.0,0.0,0.0,0.0,0.0,False,0.0,False,0.0,증상없음
5,sub-0006_ses-01,10.0,10.0,True,10.0,True,10.0,0.0,0.0,0.0,10.0,False,0.0,False,0.0,증상없음
6,sub-0007_ses-01,10.0,10.0,True,10.0,True,0.0,0.0,0.0,0.0,20.0,False,10.0,False,0.0,숨이찬느낌|질식할것같은느낌
7,sub-0008_ses-02,0.0,10.0,True,10.0,True,10.0,10.0,0.0,0.0,0.0,False,0.0,False,0.0,숨이찬느낌
8,sub-0009_ses-01,0.0,0.0,True,10.0,True,10.0,10.0,0.0,0.0,0.0,False,0.0,False,0.0,증상없음
9,sub-0011_ses-02,80.0,0.0,True,10.0,True,20.0,20.0,10.0,10.0,20.0,False,30.0,False,20.0,가슴두근거림|숨이찬느낌


In [43]:
HRV_Time_df = pd.DataFrame()
for i in T_list:
    a = G_HRV_Time_dic (path, i)
    HRV_Time_df = HRV_Time_df.append(a,ignore_index=True)
    HRV_Time_df

In [44]:
HRV_Time_df

,id,ST_base,ST_PrEx,ST_Ex2,ST_Ex6,ST_RR,ST_MR,ST_HV,ST_HD
0,sub-0001_ses-01,11:40:41.678,11:44:03.142,11:46:47.579,11:49:28.663,11:52:49.847,11:55:26.736,11:59:31.197,12:02:55.703
1,sub-0002_ses-01,12:21:49.304,12:25:07.871,12:27:53.580,12:30:35.242,12:34:01.136,12:36:34.916,12:40:30.808,12:43:57.482
2,sub-0003_ses-01,11:00:09.861,11:03:34.326,11:06:38.404,11:09:21.265,11:12:59.498,11:15:35.172,11:19:40.875,11:23:05.603
3,sub-0004_ses-01,11:58:24.105,12:01:28.666,12:04:13.049,12:06:55.989,12:09:59.244,12:12:33.710,12:16:13.816,12:19:35.068
4,sub-0005_ses-01,11:57:37.578,12:01:01.024,12:03:48.577,12:06:31.053,12:10:08.582,12:12:42.692,12:16:44.687,12:20:05.903
5,sub-0006_ses-01,12:33:42.270,12:37:10.205,12:39:58.322,12:42:43.515,12:46:22.492,12:48:59.000,12:53:01.723,12:56:26.070
6,sub-0007_ses-01,02:16:16.639,02:19:46.497,02:22:34.339,02:25:20.079,02:29:43.703,02:32:18.499,02:36:57.191,02:40:27.408
7,sub-0008_ses-02,11:44:00.293,11:47:16.566,11:50:02.132,11:52:46.941,11:56:13.175,11:58:47.984,12:02:45.011,12:06:07.096
8,sub-0009_ses-01,02:51:48.397,02:55:10.681,02:57:56.577,03:00:40.743,03:04:12.980,03:06:47.811,03:10:43.438,03:14:08.160
9,sub-0011_ses-02,11:16:06.584,11:19:28.982,11:22:18.648,11:25:02.727,11:28:38.135,11:31:15.686,11:35:18.806,11:38:45.878


In [45]:
HRV_R_df = pd.DataFrame()
for i in T_list:
    a = G_HRV_R_dic(path, i)
    HRV_R_df = HRV_R_df.append(a,ignore_index=True)
    HRV_R_df

In [46]:
HRV_R_df

,id,HRV_R_base,HRV_R_PrEx,HRV_R_Ex2,HRV_R_Ex6,HRV_R_RR,HRV_R_HV,HRV_R_HD
0,sub-0001_ses-01,0 117.344 1 117.25 2 ...,0 120.77 1 120.75 2 ...,0 121.447 1 121.25 2 ...,0 121.022 1 121 2 ...,0 120.197 1 120 2 ...,0 28.628 1 28.5 2 ...,0 28.235 1 28 2 ...
1,sub-0002_ses-01,0 117.654 1 117.5 2 ...,0 121.212 1 121 2 ...,0 121.058 1 121 2 ...,0 120.722 1 120.5 2 ...,0 119.956 1 119.75 2 ...,0 29.506 1 29.5 2 ...,0 28.729 1 28.5 2 ...
2,sub-0003_ses-01,0 118.722 1 118.5 2 ...,0 121.665 1 121.5 2 ...,0 121.799 1 121.75 2 ...,0 121.75 1 121.75 2 ...,0 120.156 1 120 2 ...,0 29.018 1 29 2 ...,0 29.228 1 29 2 ...
3,sub-0004_ses-01,0 118.222 1 118 2 ...,0 121.809 1 121.75 2 ...,0 121.606 1 121.5 2 ...,0 121.734 1 121.5 2 ...,0 119.691 1 119.5 2 ...,0 29.009 1 29 2 ...,0 28.918 1 28.75 2 ...
4,sub-0005_ses-01,0 117.927 1 117.75 2 ...,0 122.349 1 122.25 2 ...,0 121.965 1 121.75 2 ...,0 122.055 1 122 2 ...,0 120.606 1 120.5 2 ...,0 29.079 1 29 2 ...,0 27.325 1 27.25 2 ...
5,sub-0006_ses-01,0 117.69 1 117.5 2 ...,0 121.801 1 121.75 2 ...,0 122.116 1 122 2 ...,0 120.726 1 120.5 2 ...,0 119.996 1 119.75 2 ...,0 28.92 1 28.75 2 ...,0 28.776 1 28.75 2 ...
6,sub-0007_ses-01,0 117.908 1 117.75 2 ...,0 121.912 1 121.75 2 ...,0 121.904 1 121.75 2 ...,0 121.898 1 121.75 2 ...,0 120.146 1 120 2 ...,0 29.548 1 29.5 2 ...,0 26.628 1 26.5 2 ...
7,sub-0008_ses-02,0 117.897 1 117.75 2 ...,0 121.04 1 121 2 ...,0 121.197 1 121 2 ...,0 121.418 1 121.25 2 ...,0 119.85 1 119.75 2 ...,0 27.835 1 27.75 2 ...,0 27.281 1 27.25 2 ...
8,sub-0009_ses-01,0 118.409 1 118.25 2 ...,0 121.436 1 121.25 2 ...,0 121.14 1 121 2 ...,0 121.762 1 121.75 2 ...,0 119.918 1 119.75 2 ...,0 28.722 1 28.5 2 ...,0 29.109 1 29 2 ...
9,sub-0011_ses-02,0 118.039 1 118 2 ...,0 121.179 1 121 2 ...,0 121.125 1 121 2 ...,0 121.626 1 121.5 2 ...,0 120.052 1 120 2 ...,0 28.978 1 28.75 2 ...,0 29.163 1 29 2 ...
